## Loading the iris dataset

In [93]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [94]:
iris = datasets.load_iris()

In [95]:
X = iris.data[:, :4] 
Y = iris.target 
feature_size = len(iris.feature_names)
print(iris.target_names)
target_size = 1

['setosa' 'versicolor' 'virginica']


## Splitting the dataset intro train and test

In [96]:
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size = 0.20,random_state = 42) 

## Learning with Context

In [97]:
from torch_mas.head import Head
from torch_mas.data import DataBuffer

model = Head(
    feature_size, 
    target_size, 
    R=0.5,
    imprecise_th=0.01,
    bad_th=0.1,
    alpha=0.1,
    memory_length=10,
    n_epochs=3
)

## Training the agents

In [98]:
import time
dataset = DataBuffer(X_train,y_train)
t = time.time()
model.fit(dataset)
tt = time.time() - t
print(f"Total training time: {tt}s")

print("Number of agents created:", model.agents.n_agents)

Total training time: 0.7925164699554443s
Number of agents created: 66


## Computing performance

In [99]:
print(type(X_test))

<class 'numpy.ndarray'>


In [100]:
import time
import torch
test_feature_size = X_test.size
t = time.time()
y_pred = model.predict(torch.from_numpy(X_test).float()).round()
y_test = torch.from_numpy(y_test).int()
tt = time.time() - t
tps = tt / test_feature_size
print(f"Total Prediction time: {tt}s for 4 samples")
print(f"Prediction time per sample: {tps}s")
print(X_test)
print(y_pred)
print(y_test)

mse = ((y_pred - y_test) ** 2).mean()
print("Mean Squared Error:", mse.detach().numpy())

Total Prediction time: 0.0050013065338134766s for 4 samples
Prediction time per sample: 4.167755444844564e-05s
[[6.1 2.8 4.7 1.2]
 [5.7 3.8 1.7 0.3]
 [7.7 2.6 6.9 2.3]
 [6.  2.9 4.5 1.5]
 [6.8 2.8 4.8 1.4]
 [5.4 3.4 1.5 0.4]
 [5.6 2.9 3.6 1.3]
 [6.9 3.1 5.1 2.3]
 [6.2 2.2 4.5 1.5]
 [5.8 2.7 3.9 1.2]
 [6.5 3.2 5.1 2. ]
 [4.8 3.  1.4 0.1]
 [5.5 3.5 1.3 0.2]
 [4.9 3.1 1.5 0.1]
 [5.1 3.8 1.5 0.3]
 [6.3 3.3 4.7 1.6]
 [6.5 3.  5.8 2.2]
 [5.6 2.5 3.9 1.1]
 [5.7 2.8 4.5 1.3]
 [6.4 2.8 5.6 2.2]
 [4.7 3.2 1.6 0.2]
 [6.1 3.  4.9 1.8]
 [5.  3.4 1.6 0.4]
 [6.4 2.8 5.6 2.1]
 [7.9 3.8 6.4 2. ]
 [6.7 3.  5.2 2.3]
 [6.7 2.5 5.8 1.8]
 [6.8 3.2 5.9 2.3]
 [4.8 3.  1.4 0.3]
 [4.8 3.1 1.6 0.2]]
tensor([[1.],
        [0.],
        [2.],
        [1.],
        [1.],
        [0.],
        [1.],
        [2.],
        [2.],
        [1.],
        [2.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [2.],
        [1.],
        [1.],
        [2.],
        [0.],
        [2.],
        [0